In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('/kaggle/input/ria-before-2025/RIA_before_2025.csv').dropna().reset_index(drop=True).reset_index()
df

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, есл..."
1,1,2020-01-01 00:54:16,2020,1,1,Ким Чен Ын пообещал скоро представить новое ст...
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили ..."
3,3,2020-01-01 12:55:30,2020,1,1,Столкновения произошли в Гонконге во время сог...
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что нак..."
...,...,...,...,...,...,...
168730,168730,2024-12-31 18:21:42,2024,12,31,Заявка на транзит российского газа через Украи...
168731,168731,2024-12-31 19:45:52,2024,12,31,\n\n▪️Российские войска нанесли групповой удар...
168732,168732,2024-12-31 21:02:19,2024,12,31,"❗️Путин заслушал доклады начальника Генштаба, ..."
168733,168733,2024-12-31 21:05:02,2024,12,31,Повторные выбросы нефтепродуктов зафиксированы...


In [3]:
# df = df.drop(columns=['multi_labels', 'hier_label']).copy()

In [ ]:
import json
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-pretrain"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",
    torch_dtype="auto",
)

In [10]:
def generate_categories(news_text):
    prompt = f"""
Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

- **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
- **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

Входные данные - словарь (dict) с ключами: 'text' (текст новости).

Текст новости: {news_text}

Ответ:
"""
    model.config.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
    outputs = model.generate(**input_ids, max_length=2048)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)

    # Извлекаем JSON из текста
    json_match = re.search(r"\{.*\}", response, re.DOTALL)
    if json_match:
        json_text = json_match.group(0)
        try:
            return json.loads(json_text)
        except json.JSONDecodeError:
            return {"multi_labels": [], "hier_label": []}  # Возвращаем пустую структуру при ошибке
    else:
        return {"multi_labels": [], "hier_label": []}


def process_first_n_rows(df, n):
    """Обрабатывает первые n строк DataFrame, добавляя категории."""

    # Создаем копию первых n строк, чтобы избежать SettingWithCopyWarning
    df_subset = df.head(n).copy()

    # Применяем функцию generate_categories к подмножеству DataFrame
    df_subset["categories"] = df_subset["text"].apply(generate_categories)
    df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
    df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
    df_subset.drop(columns=["categories"], inplace=True)

    return df_subset

N = 10
df_processed = process_first_n_rows(df, N)

df = df.drop(index=df.index[:N])  #удаляем старые N строк, которые мы обработали
df = pd.concat([df_processed, df]).reset_index(drop=True) #объединяем обработанные строки с остальными

df

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

- **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
- **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

Входные данные - словарь (dict) с ключами: 'text' (текст новости).

Текст новости: Трамп вновь пригрозил последствиями Ирану, если кто-то из американцев в Ираке пострадает от нападений: «Иран понесет полную ответственность за гибель людей или ущерб, причиненный какому-либо из наших объектов. Они заплатят очень большую цену! Это не предупреждение, а угроза. Счастливого нового года!»

Ранее протестующие проникли за внешний периметр ограды посольства США в Багдаде. США обвинили в нападении власти Ирана, Тегеран это отрицает.

Ответ:
```json
{
  "multi_labels": ["Политика

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

- **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
- **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

Входные данные - словарь (dict) с ключами: 'text' (текст новости).

Текст новости: Ким Чен Ын пообещал скоро представить новое стратегическое оружие КНДР и заявил, что будет продолжать разработку стратегического оружия, пока враждебная политика США не будет устранена

Ответ:
```json
{
  "multi_labels": ["Политика", "Международные отношения"],
  "hier_label": ["Политика", "Внешняя политика"]
}
```


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

- **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
- **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

Входные данные - словарь (dict) с ключами: 'text' (текст новости).

Текст новости: Украинские силовики заявили, что в новом году , ранен военный.

Несмотря на то что "нормандские лидеры" призывали до конца 2019 года обеспечить прекращение огня.

Ответ:

```json
{
  "multi_labels": ["Происшествия"],
  "hier_label": ["Происшествия", "Происшествия"]
}
```


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

- **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
- **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

Входные данные - словарь (dict) с ключами: 'text' (текст новости).

Текст новости: Экс-глава Национальной баскетбольной ассоциации Дэвид Стерн скончался в среду в возрасте 77 лет.

Спортивный менеджер встал у руля NBA в 1984 году и возглавлял ее три десятка лет, сделав одной из самых богатых спортивных лиг в мире - третьей в Северной Америке после NFL (американский футбол) и MLB (бейсбол). В декабре Стерн был госпитализирован из-за кровоизлияния в мозг.

Ответ:
```json
{
  "multi_labels": ["Спорт"],
  "hier_label": ["Спорт", "Баскетбол"]
}
```

Ты — редактор новостног

,index,date,year,month,day,text,multi_labels,hier_label
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, если кто-то из американцев в Ираке пострадает от нападений: «Иран понесет полную ответственность за гибель людей или ущерб, причиненный какому-либо из наших объектов. Они заплатят очень большую цену! Это не предупреждение, а угроза. Счастливого нового года!»\n\nРанее протестующие проникли за внешний периметр ограды посольства США в Багдаде. США обвинили в нападении власти Ирана, Тегеран это отрицает.","[Политика, Происшествия]","[Политика, Внешняя политика]"
1,1,2020-01-01 00:54:16,2020,1,1,"Ким Чен Ын пообещал скоро представить новое стратегическое оружие КНДР и заявил, что будет продолжать разработку стратегического оружия, пока враждебная политика США не будет устранена","[Политика, Международные отношения]","[Политика, Внешняя политика]"
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили Новый год, участвуя в праздничных мероприятиях, сообщило МВД РФ. Нарушений правопорядка не допущено. В МВД добавили, что общественный порядок охраняли около 67,4 тысячи полицейских.","[Происшествия, Общество]","[Происшествия, Общественный порядок]"
3,3,2020-01-01 12:55:30,2020,1,1,"Столкновения произошли в Гонконге во время согласованной акции, полиция применила слезоточивый газ для разгона протестующих, сообщает газета South China Morning Post.","[Происшествия, Международные отношения]","[Происшествия, Международные отношения]"
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что накануне отшлепал по руке одну не в меру активную паломницу.\n\nВо вторник вечером женщина не на шутку рассердила папу Франциска, который вышел на площадь Святого Петра в Ватикане, чтобы полюбоваться установленными там под елью рождественскими яслями и поприветствовать верующих. Понтифик по обычаю пожимал присутствующим руки и уже собирался вернуться в папские покои, когда паломница неожиданно схватила его за правую руку и бесцеремонно потянула ее к себе.\n\n",[],[]
...,...,...,...,...,...,...,...,...
168730,168730,2024-12-31 18:21:42,2024,12,31,"Заявка на транзит российского газа через Украину на 1 января оказалась нулевой, свидетельствуют данные оператора ГТС Украины",NaN,NaN
168731,168731,2024-12-31 19:45:52,2024,12,31,"\n\n▪️Российские войска нанесли групповой удар по военному аэродрому и производству пороховых зарядов для ВСУ, заявили в Минобороны; \n\n▪️Российские войска нанесли поражение противнику и отразили 15 контратак, потери Киева составили до 1495 военнослужащих, добавили в ведомстве; \n\n▪️На курском направлении ВСУ за сутки потеряли более 445 военнослужащих, за всё время - до 45980 человек, говорится в отдельной сводке; \n\n▪️Эрдоган объявил, что приоритетом страны в 2025 году станет заключение справедливого мира между Россией и Украиной; \n\n▪️Почти все опрошенные ООН российские пленные на Украине заявили, что подвергались пыткам, свидетельствует доклад ООН; \n\n▪️ВСУ добивали некоторых российских солдат FPV-дронами, это военное преступление, говорится в докладе ООН; \n\n▪️Российские войска нанесли удары по технике и военным ВСУ, складу с оружием и корпусу с расчетом БПЛА в Шостке, ракетному производству в Павлограде, сообщили РИА Новости в пророссийском подполье.",NaN,NaN
168732,168732,2024-12-31 21:02:19,2024,12,31,"❗️Путин заслушал доклады начальника Генштаба, а также командующих группировок о ходе спецоперации и тепло поздравил с наступающим Новым годом всех военнослужащих и участников СВО, сообщил Песков",NaN,NaN
168733,168733,2024-12-31 21:05:02,2024,12,31,"Повторные выбросы нефтепродуктов зафиксированы на побережье Анапы. Власти города просят не подходить близко к воде, очистка берега начнется утром.",NaN,NaN


In [5]:
# import json
# import re
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd

# # --- Модифицированный код для модели T-lite ---
# MODEL_NAME = "t-tech/T-lite-it-1.0"

# # Инициализация модели и токенизатора
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="auto",
#     torch_dtype="auto",
# )

# def generate_categories(news_text):
#     # Формируем системный промпт и запрос пользователя
#     messages = [
#         {
#             "role": "system",
#             "content": """Ты — редактор новостного агрегатора в России. Твоя задача: категоризировать новости для российских читателей, создав две колонки: multi_labels и hier_label.
            
#             multi_labels: Список [,] из наиболее значимых категорий новости.
#             hier_label: Список [,] - Двухуровневая иерархия для первой категории.
            
#             Необходим ответ без пояснений строго в формате JSON."""
#         },
#         {
#             "role": "user", 
#             "content": f"Текст новости: {news_text}"
#         }
#     ]
    
#     # Применяем чатовый шаблон
#     text = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )
    
#     # Токенизация и генерация
#     model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
#     generated_ids = model.generate(
#         **model_inputs,
#         max_new_tokens=128,
#         pad_token_id=tokenizer.eos_token_id
#     )
    
#     # Извлекаем сгенерированный текст
#     generated_ids = [
#         output_ids[len(input_ids):] 
#         for input_ids, output_ids 
#         in zip(model_inputs.input_ids, generated_ids)
#     ]
#     response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print(response)
#     # Парсинг JSON
#     try:
#         json_match = re.search(r"\{.*\}", response, re.DOTALL)
#         return json.loads(json_match.group(0)) if json_match else {"multi_labels": [], "hier_label": []}
#     except json.JSONDecodeError:
#         return {"multi_labels": [], "hier_label": []}

# # Остальной код обработки DataFrame остается без изменений
# def process_first_n_rows(df, n):
#     df_subset = df.head(n).copy()
#     df_subset["categories"] = df_subset["text"].apply(generate_categories)
#     df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
#     df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
#     df_subset.drop(columns=["categories"], inplace=True)
#     return df_subset

# # Параметры обработки
# N = 10  # Сколько строк обрабатывать за раз

# # Основной цикл обработки
# df_processed = process_first_n_rows(df, N)
# print("Обработанные данные:")
# print(df_processed)

# # Обновление основного DataFrame (как в твоем оригинальном коде)
# df = df.drop(index=df.index[:N])
# df = pd.concat([df_processed, df]).reset_index(drop=True)
# df

In [6]:
pd.set_option('display.max_colwidth', None)

In [7]:
df.head(10)

,index,date,year,month,day,text,multi_labels,hier_label
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, если кто-то из американцев в Ираке пострадает от нападений: «Иран понесет полную ответственность за гибель людей или ущерб, причиненный какому-либо из наших объектов. Они заплатят очень большую цену! Это не предупреждение, а угроза. Счастливого нового года!»\n\nРанее протестующие проникли за внешний периметр ограды посольства США в Багдаде. США обвинили в нападении власти Ирана, Тегеран это отрицает.","[Политика, Происшествия]","[Политика, Международные отношения]"
1,1,2020-01-01 00:54:16,2020,1,1,"Ким Чен Ын пообещал скоро представить новое стратегическое оружие КНДР и заявил, что будет продолжать разработку стратегического оружия, пока враждебная политика США не будет устранена","[Политика, КНДР, Ядерное оружие]","[Политика, КНДР]"
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили Новый год, участвуя в праздничных мероприятиях, сообщило МВД РФ. Нарушений правопорядка не допущено. В МВД добавили, что общественный порядок охраняли около 67,4 тысячи полицейских.","[Праздники, Безопасность]","[Праздники, Новый год]"
3,3,2020-01-01 12:55:30,2020,1,1,"Столкновения произошли в Гонконге во время согласованной акции, полиция применила слезоточивый газ для разгона протестующих, сообщает газета South China Morning Post.","[Гонконг, Протесты, Полиция]","[Гонконг, Протесты]"
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что накануне отшлепал по руке одну не в меру активную паломницу.\n\nВо вторник вечером женщина не на шутку рассердила папу Франциска, который вышел на площадь Святого Петра в Ватикане, чтобы полюбоваться установленными там под елью рождественскими яслями и поприветствовать верующих. Понтифик по обычаю пожимал присутствующим руки и уже собирался вернуться в папские покои, когда паломница неожиданно схватила его за правую руку и бесцеремонно потянула ее к себе.\n\n","[Скандалы, Религия]","[Религия, Скандалы]"
5,5,2020-01-01 15:45:51,2020,1,1,"Более 75,5 тысяч пассажиров встретили Новый год в поездах дальнего следования, сообщила ""Федеральная пассажирская компания""","[Новый год, Пассажиры, Поезда дальнего следования]","[Праздники, Новый год]"
6,6,2020-01-01 17:12:14,2020,1,1,"В России в ночь на 1 января , сообщает «Столото». Теперь счастливчика ждут для оформления выигрыша. Пока известно лишь то, что билет был куплен в Москве.","[lottery, russia, new year]","[lottery, russia]"
7,7,2020-01-01 19:38:39,2020,1,1,"Украинские силовики заявили, что в новом году , ранен военный.\n\nНесмотря на то что ""нормандские лидеры"" призывали до конца 2019 года обеспечить прекращение огня.","[Война и конфликты, Украина]","[Война и конфликты, Украина]"
8,8,2020-01-02 00:51:53,2020,1,2,"Экс-глава Национальной баскетбольной ассоциации Дэвид Стерн скончался в среду в возрасте 77 лет.\n\nСпортивный менеджер встал у руля NBA в 1984 году и возглавлял ее три десятка лет, сделав одной из самых богатых спортивных лиг в мире - третьей в Северной Америке после NFL (американский футбол) и MLB (бейсбол). В декабре Стерн был госпитализирован из-за кровоизлияния в мозг.","[Спорт, Баскетбол]","[Спорт, Баскетбол]"
9,9,2020-01-02 03:52:11,2020,1,2,"МЧС хочет из зон чрезвычайных ситуаций при соответствующем решении властей.\n\nВ законопроекте, в частности, предусмотрены ""возможность принудительной эвакуации населения из зон ЧС"" и ""введение ответственности за несвоевременное представление информации о ЧС"".","[ЧС, Эвакуация]","[ЧС, Эвакуация]"
